# Auto Registration

This is a *OpenCV* and *Gdal* for autoregistration of 2 images

---

1. Load Images and Libraries

1.1 Checks raw bit size 8

In [7]:
import cv2
import numpy as np
import rasterio

path_location = "D:/1. PROJECT/satellite_etl/satellite/test3/"

def register_images(template_path, reference_path, output_path):
    """Register template image to reference image using SIFT + RANSAC homography."""

    # Load reference (georeferenced)
    ref_ds = rasterio.open(reference_path)
    ref_img = ref_ds.read(1)
    ref_profile = ref_ds.profile

    # Load template
    temp_ds = rasterio.open(template_path)
    temp_img = temp_ds.read(1)

    # Normalize to 8-bit grayscale for SIFT
    def normalize_to_uint8(arr):
        arr = arr.astype(np.float32)
        arr = 255 * (arr - arr.min()) / (arr.max() - arr.min())
        return arr.astype(np.uint8)

    ref_gray = normalize_to_uint8(ref_img)
    temp_gray = normalize_to_uint8(temp_img)

    # Feature detection
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(temp_gray, None)
    kp2, des2 = sift.detectAndCompute(ref_gray, None)

    print(f"Template keypoints: {len(kp1)}, Reference keypoints: {len(kp2)}")

    # Matching
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    # Lowe’s ratio test
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    print(f"Good matches: {len(good_matches)}")

    if len(good_matches) < 10:
        raise RuntimeError("Not enough matches for homography.")

    # Extract matched points
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Estimate homography
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    print("Homography:\n", H)

    # Warp template
    h, w = ref_gray.shape
    warped = cv2.warpPerspective(temp_img, H, (w, h))

    # Save registered image with reference georeferencing
    out_profile = ref_profile.copy()
    out_profile.update({"driver": "GTiff", "count": 1, "dtype": warped.dtype})

    with rasterio.open(output_path, "w", **out_profile) as dst:
        dst.write(warped, 1)

    print(f"Registered image saved to {output_path}")
    return H


if __name__ == "__main__":
    template = path_location + "sss.tif"
    reference = path_location + "oldimage.tif"
    output = path_location + "georegistered.tif"

    H = register_images(template, reference, output)


Template keypoints: 1260, Reference keypoints: 1393
Good matches: 26
Homography:
 [[-1.93856594e+00  6.08145657e-01  2.33107220e+03]
 [-1.60169119e+00  4.98250670e-01  1.92929268e+03]
 [-8.31047264e-04  2.59789599e-04  1.00000000e+00]]
Registered image saved to D:/1. PROJECT/satellite_etl/satellite/test3/georegistered.tif
